In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110540

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,211534.1422
2018-02-28,184856.0803
2018-03-31,185494.3785
2018-04-30,187746.4373
2018-05-31,196964.2830
2018-06-30,195731.7027
2018-07-31,204901.3312
2018-08-31,214606.9070
2018-09-30,175722.4742


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    211534.1422
 2018-02-28    184856.0803
 2018-03-31    185494.3785
 2018-04-30    187746.4373
 2018-05-31    196964.2830
 2018-06-30    195731.7027
 2018-07-31    204901.3312
 2018-08-31    214606.9070
 2018-09-30    175722.4742
 2018-10-31    182742.0300
 2018-11-30    192747.9371
 2018-12-31    185034.0514
 2019-01-31    236999.3358
 2019-02-28    202048.8710
 2019-03-31    228706.9648
 2019-04-30    195831.5571
 2019-05-31    229946.2458
 2019-06-30    229627.3713
 2019-07-31    282299.4561
 2019-08-31    255234.3618
 2019-09-30    244019.2931
 2019-10-31    246469.1895
 2019-11-30    232955.4488
 2019-12-31    227492.3911
 2020-01-31    233755.4151
 2020-02-29    219395.4597
 2020-03-31    248951.4876
 2020-04-30    233106.2583
 2020-05-31    235492.7259
 2020-06-30    210735.0150
 2020-07-31    241015.6560
 2020-08-31    216136.0976
 2020-09-30    220084.2651
 2020-10-31    219095.1371
 2020-11-30    212189.5622
 2020-12-31    218060.9411

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.936969
p-value : 0.314874
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -12.709063
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=967.486, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=964.430, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=952.741, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=951.512, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=954.630, Time=0.09 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=958.388, Time=0.09 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=957.436, Time=0.06 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=953.100, Time=0.10 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=952.314, Time=0.15 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=952.982, Time=0.38 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=949.443, Time=0.09 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=950.629, Time=0.16 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=952.674, Time=0.11 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=953.263, Time=0.20 sec
 ARIMA(3,0,0)(0,0,0)[1] inter

ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 0)   Log Likelihood                -470.721
Method:                       css-mle   S.D. of innovations          17666.274
Date:                Sat, 11 Sep 2021   AIC                            949.443
Time:                        04:03:02   BIC                            956.394
Sample:                             0   HQIC                           951.991
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.156e+05   8922.826     24.163      0.000    1.98e+05    2.33e+05
ar.L1.y        0.2307      0.133      1.736      0.083      -0.030       0.491
ar.L2.y        0.4909      0.137      3.592      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([224018.01525931, 211817.16496161]), array([17666.27402671, 18130.22522307]), array([[189392.75442595, 258643.27609268],
       [176282.57649279, 247351.75343043]]))
